### 2.1 assert
assert выражение, "текст_ошибки"

Инструкция assert предназначена не для ошибок времени исполнения программы, а для отладки ошибок, которые не должны происходить вовсе

In [35]:
# правильный пример использования №1
def apply_discount(product, discount):
    price = int(product['price'] * (1.0 - discount))
    assert 0 <= price <= product['price'], ('Из-за скидки, цена стала меньше нуля или больше цены продукта')
    return price

shoes = {'name': 'fancy shoes', 'price': 14900}
apply_discount(shoes, 0.25)

11175

assert, также нельзя использовать для проверки данных, т.к. инструкции assert могут быть глобально отключены

In [36]:
# неправильный пример использования №2
def delete_product(prod_id, user):
    assert user.is_admin(), 'здесь должен быть администратор'
    assert store.has_product(prod_id), 'Неизвестный товар'
    store.get_product(prod_id).delete()

# правильный пример использования №2
def delete_product(prod_id, user):
    if not user.is_admin(): 
        raise AuthError("Для удаления необходимы права админа")
    if not store.has_product(prod_id): 
        raise ValueError("Идентификатор неизвестного товара")
    store.get_product(prod_id).delete()

Когда в инструкцию assert в качестве первого аргумента передается непустой кортеж, то она всегда возвращает True, а значит становится бесполезной

In [37]:
# неправильный пример использования №3
assert(
    1==2, 
    'Это утверждение не вызовет сбой'
)

<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
C:\Temp\ipykernel_8320\1530422251.py:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(


### 2.2 Запятые

In [38]:
# формат, который не советуется использовать 
names = ['Элис', 'Боб', 'Дилберт']

# формат, который советуется использовать 
names = [
    'Элис', 
    'Боб', 
    'Дилберт'
]

конкатинация строковых литералов

In [39]:
my_str = ('Это супердлинная строковая константа, '
          'развернутая на несколько строк. '
          'И обратите внимание - не требует никаких \ !')
print(my_str)

Это супердлинная строковая константа, развернутая на несколько строк. И обратите внимание - не требует никаких \ !


В python запятую можно ставить после любого элемента, даже последнего

In [40]:
names = [
    'Элис', 
    'Боб', 
    'Дилберт',
]

### 2.3 Менеджеры контекста и инструкция with


In [41]:
# Пример применения №1
with open('hello.txt', 'w') as f:
    f.write('Hello world')

# Данный фрагмент примерно сводится к 

f = open('hello.txt', 'w')
try:
    f.write('Hello world')
finally:
    f.close()

In [42]:
# Пример применения №1
import threading

some_lock = threading.Lock()
with some_lock:
    pass

# Данный фрагмент примерно сводится к
some_lock = threading.Lock()
some_lock.acquire()
try:
    pass
finally:
    some_lock.release()

#### Поддержка инструкции with в собственных объектах
Python вызывает enter, когда поток исполнения входит в контекст инструкции with и наступает момент получения ресурса.
Когда поток исполнения снова покидает контекст, python вызывает exit, чтобы освободить этот ресурс

In [43]:
class ManagedFile:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

with ManagedFile('hello.txt') as f:
    f.write("Hello world!")
    f.write("And goodbye")

Вариант с contextlib

В данном случае managed_file является генератором

In [44]:
from contextlib import contextmanager

@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()

with managed_file('hello.txt') as f:
    f.write("Hello world!")
    f.write("And goodbye")

#### Написание API с менеджером контекста

In [45]:
class Indenter:
    def __init__(self):
        self.level = 0

    def __enter__(self):
        self.level += 1
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level -= 1

    def print(self, text):
        print(' ' * self.level + text)

with Indenter() as indent:
    indent.print('Hello')
    with indent:
        indent.print("What's up")
        with indent:
            indent.print("Bonjur")
    indent.print('hey')

 Hello
  What's up
   Bonjur
 hey


### 2.4 Подчеркивания, дандеры и др.
#### 1. Одинарный начальный символ подчеркивания _var
На поведение программ не влияет и является лишь подсказкой о том, что переменая предназначается для внутреннего использования

In [46]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 23

t = Test()
t.foo # Доступ к публичному атрибуту
t._bar  # Доступ к защищенному атрибуту, но это не рекомендуется делать вне класса

23

Если нижнее начальное подчеркивание использовано в модуле, то при подстановочном импорте * Python не будет испортировать имена с нижним подчеркиванием в начале, если не определен список ```__all__``` или не использован обычный импорт

In [47]:
# my_module.py
def external_func():
    return 23

def _internal_func():
    return 42

In [ ]:
# Неверный пример использования импорта
from my_module import *
external_func() 
_internal_func()  # Это вызовет ошибку, так как _internal_func не импортирован из my_module

# Верный пример использования импорта
import my_module
my_module.external_func()  # Доступ к публичной функции
my_module._internal_func()  # Доступ к защищенной функции, но это не рекомендуется

#### 2. Одинарный замыкающий символ подчеркивания var_
Используется по договоренности, чтобы избежать конфликтов из-за совпадения имен с ключевыми словами Python

In [ ]:
def make_object(name, class_):
    pass

#### 3. Двойной начальный символ подчеркивания __var
Данный префикс, заставляет интерпретатор Python переписывать имя атрибута, для того, чтобы в подклассах избежать конфликтов из-за совпадения имен. В dir от класса, данный метод будет назван _Classname__methodname

In [ ]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 23
        self.__baz = 23 # Данный атрибут будет переименован в _Test__baz
    
class ExtendedTest(Test):
    def __init__(self):
        super().__init__()
        self.foo = "переопределено"
        self._bar = "переопределено" 
        self.__baz = "переопределено"

t2 = ExtendedTest()
t2.foo 
t2._bar 
# t2.__baz  # Это вызовет ошибку, так как __baz переименован в _ExtendedTest__baz
t2._ExtendedTest__baz # Доступ к переименованному атрибуту будет возможен

'переопределено'

Доступ к переименованному элементу можно произвести с помощью метода, возвращающего данный элемент

In [ ]:
class ManglingTest:
    def __init__(self):
        self.__mangled = "Привет"
    
    def get_mangled(self):
        return self.__mangled

ManglingTest().get_mangled() # Доступ к защищенному атрибуту через метод класса
ManglingTest().__mangled # Это вызовет ошибку, так как __mangled переименован в _ManglingTest__mangled

Можно задать переименованный элемент глобальной переменной

In [ ]:
_MangledGlobal__mangled = 23

class MangledGlobal:
    def test(self):
        return __mangled 

MangledGlobal().test()  # Это вернет 23 и не вызовет ошибку

23

#### Экскурс: что такое дандеры

Дандер - dunder - double underscore

__baz - дандер baz

```__init__``` - дандер init

#### Двойной начальный и замыкающий символ подчеркивания ```__var__```
Доступ к подобным переменным возможен, как обычно, однако не рекомендуется использовать такие имена, так как они могут конфликтовать с именами встроенных методов Python

In [ ]:
class PrefixPostfixTest:
    def __init__(self):
        self.__bam__ = 42

PrefixPostfixTest().__bam__ 

42

#### 5. Одинарный символ подчеркивания: _
По договоренности одинарный автономный символ подчеркивания, используется в качестве имени, чтобы подчеркнуть, что эта переменная временная или незначительная.

In [ ]:
for _ in range(5):
    print("Hello, World!")

car = ("красный", "легковой автомобиль", 12, 3812.4)
color, _, _, mileage = car  # Использование _ для незначительных значений

Помимо этого символ _ можно использовать для вывода результата последнего выражения, вычисленного интерпретатором REPL Python

```
>>> 20 + 3
23
>>> _
23
>>> print(_)
23
```


```
>>> list()
[]
>>> _.append(1)
>>> _.append(2)
>>> _.append(3)
>>> _
[1, 2, 3]
```

### 2.5 Форматирование строковых значений

In [2]:
errno = 50159747054
name = 'Боб'

# хотим сгенерировать строку
# Эй, Боб! Вот ошибка 0xbadc0ffee!


#### №1. "Классическое" форматирование строковых значений
Есть встроенные операции через оператор %

%s - подставить значение переменной в виде строкового

In [3]:
'Привет, %s' % name

'Привет, Боб'

%x - представить число в шестнадцатеричной форме

In [4]:
'%x' % errno  # Представление числа в шестнадцатеричной форме 

'badc0ffee'

In [10]:
'Эй, %s! Вот ошибка 0x%x!' % (name, errno) 

'Эй, Боб! Вот ошибка 0xbadc0ffee!'

In [11]:
'Эй, %(name)s! Вот ошибка 0x%(errno)x!' % {'name': name, 'errno': errno} 

'Эй, Боб! Вот ошибка 0xbadc0ffee!'

#### №2. "Современное" форматирование строковых значений

In [5]:
'Привет, {}'.format(name)

'Привет, Боб'

In [6]:
'Эй, {name}! Вот ошибка 0x{errno:x}!'.format(name=name, errno=errno)

'Эй, Боб! Вот ошибка 0xbadc0ffee!'

#### №3. Интерполяция литеральных строк (Python 3.6+)

In [7]:
f'Привет, {name}'

'Привет, Боб'

In [9]:
a = 5
b = 10
f'Пять плюс десять равняется {a + b}, а не {2 * (a + b)}'

'Пять плюс десять равняется 15, а не 30'

In [10]:
def greet(name, question):
    return f'Привет, {name}! Как {question}?'

greet('Боб', 'дела') # Пример вызова функции с интерполяцией строки

'Привет, Боб! Как дела?'

In [13]:
f"Эй, {name}! Вот ошибка {errno:#x}!" 

'Эй, Боб! Вот ошибка 0xbadc0ffee!'

#### №4 Шаблонные строки

In [ ]:
from string import Template
t = Template('Эй, $name!')
t.substitute(name=name)  # Использование шаблонной строки для подстановки переменной

'Эй, Боб!'

In [17]:
templ_string = 'Эй, $name! Вот ошибка $error!'
Template(templ_string).substitute(name=name, error=hex(errno))  # Подстановка переменных в шаблонную строку

'Эй, Боб! Вот ошибка 0xbadc0ffee!'

Форматные строки могут получать доступ к произвольным переменным в программе

In [20]:
SECRET = 'это - секретная строка'
class Error:
    def __init__(self):
        pass

err = Error()
user_input = '{error.__init__.__globals__[SECRET]}'
user_input.format(error=err) # Доступ к глобальной переменной в шаблонной строке

'это - секретная строка'

In [ ]:
user_input = '${error.__init__.__globals__[SECRET]}'
Template(user_input).substitute(error=err)  
# Шаблонная строка не позволяет получать доступ к глобальным переменным напрямую

Если форматирующие строки поступают от пользователей, то используйте шаблонные строки, чтобы избежать проблем с безопасностью. В противном случае используйте интерполяцию литеральных строк при условии, что вы работаете с Python 3.6+ и современное форматирование строк - если нет

### 2.6 Пасхалка "Дзен Python"

In [21]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
